In [11]:
import sys, os
import pathlib
import time
import importlib
import joblib
import pickle
import shutil
from tqdm import tqdm

import numpy as np

from scipy.linalg import null_space
from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt

import cv2

In [2]:
%load_ext autoreload
%autoreload 2

if not os.path.abspath("../") in sys.path:
    sys.path.append(os.path.abspath("../"))
    sys.path.append(os.path.abspath("../src"))

from src import normalization
from src import Camera, metashape_Camera
from src import camera_correspondence, FF_mat, epipole_angle, pair_and_key_gen, coll_dict_gen, pair_pt_gen
from src import FR_check, coordinate_dict_gen
from src import min_dist, tri, TDlines_gen, reprojection_gen, gen_support_dict, gen_support
from src import all_D, frag_list_fraged, all_sep

In [3]:
def all_prop(img_num, img_folder, label_path, view_mat_folder, output):
    # 画像の読み込み
    cam_list = [Camera(i) for i in range(img_num)]
    for i in range(len(cam_list)):
        cam_list[i].img_load(img_folder)
        cam_list[i].contour_extraction()
        cam_list[i].label_load(label_path)
        cam_list[i].correspondence_contour()
        cam_list[i].para_load("{}/{}.csv".format(view_mat_folder, str(i)))
    
    # 画像の加工
    kernel = np.ones((3,3),np.uint8)
    w = W
    for cnum in range(len(cam_list)):
        sparse_mats_list = []
        for j in range(len(cam_list[cnum].contour_list)):
            new_img = np.zeros((cam_list[0].img[0].shape[0],cam_list[0].img[0].shape[1]),dtype=np.uint8)
            for i in range(len(cam_list[cnum].contour_list[j])):
                curve = cam_list[cnum].contour_list[j][i][~np.isnan(cam_list[cnum].contour_list[j][i])].reshape((-1,2)).astype(int)
                new_img[curve[:,1],curve[:,0]]=True
            dilation = cv2.dilate(new_img, kernel, iterations = w)
            new_img = csr_matrix(dilation, dtype=np.uint8)
            sparse_mats_list.append(new_img)
        cam_list[cnum].contour_img = sparse_mats_list
    
    # カメラペアの作成
    b = B
    cam_pairs = camera_correspondence(cam_list,angle_upper=b / 180 * np.pi)
    print(cam_pairs)
    # カメラペア間のエピポール，基礎行列取得
    epipole_dict = {i:[] for i in range(len(cam_list))}
    cam_pairs_F = {}
    for i in cam_pairs:
        epipole1, epipole2, F = FF_mat(cam_list[i[0]].A, cam_list[i[1]].A, cam_list[i[0]].Rt, cam_list[i[1]].Rt)
        epipole_dict[i[0]].append(normalization(epipole1))
        epipole_dict[i[1]].append(normalization(epipole2))
        cam_pairs_F[i] = F

    
    target_dir = output

    shutil.rmtree(target_dir)
    os.mkdir(target_dir)
    
    # フラグメントのリストを作る
    for i in range(len(cam_list)):
        im_del_list = all_D(epipole_angle(i, epipole_dict, cam_list=cam_list))# im_del_list[color][contour][del_idx]
        newCon = all_sep(cam_list[i].contour_list, im_del_list)# newCon[color][fragment][coordination]
        cam_list[i].frag_list = frag_list_fraged(newCon)
    del epipole_dict
    
    _ = joblib.Parallel(n_jobs=-1, verbose=0)(joblib.delayed(pair_and_key_gen)(
        i, cam_list, cam_pairs_F, dest_dir=output) for i in cam_pairs_F)
    temp_path = pathlib.Path(output)
    tags_path = list(temp_path.glob("*.pair_list"))

    tags = []
    for tag_path in tags_path:
        tag_sp = tag_path.stem.split("_")
        tag_arr = ((int(tag_sp[0]), int(tag_sp[1])), tag_sp[2])
        tags.append(tag_arr)
    
    _ = joblib.Parallel(n_jobs=-1, verbose=0)(joblib.delayed(coll_dict_gen)(
        i, cam_list, cam_pairs_F, dest_dir=output) for i in tags)
    
    _ = joblib.Parallel(n_jobs=-1, verbose=0)(joblib.delayed(pair_pt_gen)(
        i, dest_dir=output) for i in tags)
    
    _ = joblib.Parallel(n_jobs=-1, verbose=0)(joblib.delayed(coordinate_dict_gen)(
        i, cam_list=cam_list, dest_dir=output) for i in tags)
    
    _ = joblib.Parallel(n_jobs=-1,verbose=0)(joblib.delayed(TDlines_gen)(
        tag, cam_list=cam_list, cam_pairs_F=cam_pairs_F, src_dir=output) for tag in tags)
    
    _ = joblib.Parallel(n_jobs=-1,verbose=0)(joblib.delayed(reprojection_gen)(
        tag, cam_list=cam_list, tmp_dir=output) for tag in tags)
    
    _ = joblib.Parallel(n_jobs=-1,verbose=0)(joblib.delayed(gen_support)(
        tag, cam_list=cam_list, dest_dir=output) for tag in tags)

In [4]:
def get_tags(temp_folder):
    tags_path = list(temp_folder.glob("*.TDlines"))

    tags = []
    for tag_path in tags_path:
        tag_sp = tag_path.stem.split("_")
        tag_arr = ((int(tag_sp[0]), int(tag_sp[1])), tag_sp[2])
        tags.append(tag_arr)
    return tags

def support_th(tags, temp_folder, sup_th):
    curve_fragment = []
    for tag in tags:
        if tag[1] == "R":
            continue

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.TDlines".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            lines_list = pickle.load(f)

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.support_dict".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            support_list, support_ac = pickle.load(f)
        #support_list, support_ac = support_dict[tag][0], support_dict[tag][1]
        tag_list = []
        for col, sup_col, ac_col in zip(lines_list, support_list, support_ac):
            col_curve_fragment = []
            for frag, sup, sup_ac in zip(col, sup_col, ac_col):
                if sup > sup_th:
                    frag = np.reshape(frag,(-1, 3))
                    frag = np.array([i for i,j in zip(frag, sup_ac >sup_th) if j])
                    col_curve_fragment.append(frag)
            tag_list.append(col_curve_fragment)
        curve_fragment.append(tag_list)
    return curve_fragment

def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry

angle = [0, np.pi/2, np.pi/2] # pcd.rotate()
angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

def curve_fragment_align(curve_fragment):
    cfs = [[] for i in range(8)]
    for cols in curve_fragment:
        for i in range(len(cols)):
            cfs[i] += cols[i]
    temp_cfs = []
    for i, cf_label in enumerate(cfs):
        A = [frag  for frag in cf_label if frag.size!=0]
        temp_cfs.append(A)
    cfs = temp_cfs
    cfs = [list(map(lambda x: x*10@R@R_mirror, cf_label)) for cf_label in cfs]
    return cfs



In [5]:
# input
img_num = 0#image number
img_folder = "IMAGE_FOLDER"
label_path = "LABEL_PATH"
view_mat_folder = "VIEW_MAT_FOLDER"
output = "OUTPUT_FOLDER"

sup_para = "SUPPORT_THRESHOLD_PARAMETER"
occ_path = "OCCLUSION_PICKLE_PATH"
supported_output_folder = "SUPPORTED_OUTPUT_FOLDER"
supported_output = "SUPPORTED_OUTPUT"

In [6]:
# demo input
img_num = 32
img_folder = "data/image/demo_image/"
label_path = "data/labels/demo_poly_label.pickle"
view_mat_folder = "data/view_mat/demo_view_mat/"
output = "data/temp"

sup_para = "data/support_th/bottom_sup_th_para.pickle"
occ_path = "data/labels/demo_poly_occlusion.pickle"
supported_output_folder = "data/supported_curve"
supported_output = "demo_supported_curves.pickle"

In [48]:
# demo input
data_name="rolling_alpha_016"
img_num = 64
img_folder = "data/image/image_"+data_name+"/"
label_path = "data/labels/64_stub_label.pickle"
view_mat_folder = "data/view_mat/view_mat/"
output = "data/"+data_name+"_temp"
sup_para = "data/support_th/bottom_sup_th_para.pickle"
#occ_path = "data/labels/demo_poly_occlusion.pickle"
th_num = 40
th = [th_num]*8
supported_output_folder = "data/supported_curve"
supported_output = str(th_num) + "_"+data_name+"_supported_curves.pickle"

In [49]:
# hyper parameter
W = 5 # サポート判定の距離
B = 60 # ベースライン


In [50]:
#all_prop(img_num, img_folder, label_path, view_mat_folder, output)

In [51]:
tags = get_tags(pathlib.Path(output))

temp_cfs = []
for i, osth in enumerate(th):
    curve_fragment = support_th(tags, output, osth)
    cfs = curve_fragment_align(curve_fragment)
    temp_cfs.append(cfs[i])
    #print(curve_fragment)

os.makedirs(supported_output_folder, exist_ok=True)
with open(os.path.join(supported_output_folder, supported_output), 'wb') as f:
    pickle.dump(temp_cfs, f)